In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
data = pd.read_csv("content_topic_prediction/Topic_prediction_data_train.txt",sep='\t',header=None)
data.columns = ['text','uid','type','timestamp']
labels = pd.read_csv("content_topic_prediction/Topic_prediction_labels_train.txt",sep='\t',header=None)
labels.columns = ['label']
data['label'] = labels['label']



In [3]:
data.head()

,text,uid,type,timestamp,label
0,distinct consubstanti translat roman missal pu...,0,Q,0.01,9
1,cathol christian power bind loos catholic beli...,0,Q,0.01,0
2,read scriptur common worship christian church ...,0,Q,0.01,9
3,accord person polit libertarian compat golden ...,0,Q,0.01,0
4,criteria cathol determin testament charact pre...,0,Q,0.01,0


In [4]:
data.shape

(98116, 5)

In [5]:
data = data[(data['text'].notnull())]

In [6]:
data=data.reset_index(drop=True)
data.shape

(96269, 5)

In [7]:
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 5000)

In [8]:
train_data_features = vectorizer.fit_transform(data["text"])
features= np.array(train_data_features)

In [132]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
classifier = LogisticRegression(random_state=1)
#classifier= GradientBoostingClassifier()
#classifier= GaussianNB()



In [135]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
# scores = cross_val_score(classifier, train_data_features, data["label"], cv=3)
x_Train, x_Test, y_Train,  y_Test= train_test_split(train_data_features,data["label"], test_size=.01)
clf= classifier.fit(x_Train,y_Train)

(95306, 5000) (963, 5000)


In [12]:
text_f=["hello this is a good day","nice day this is", "hello how are you", "hello how are you today", "batman wins all", "hey how are you"]
corpus = pd.DataFrame(columns=["ftext"])
corpus["ftext"] = text_f


In [63]:
corpus

,ftext
0,hello this is a good day
1,nice day this is
2,hello how are you
3,hello how are you today
4,batman wins all
5,hey how are you


In [70]:
kkl=corpus.drop([2])
kkl = kkl.reset_index(drop=True)
kkl

,ftext
0,hello this is a good day
1,nice day this is
2,hello how are you today
3,batman wins all
4,hey how are you


In [69]:
kkl.iloc[2]

ftext    hello how are you today
Name: 3, dtype: object

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfVect= TfidfVectorizer(min_df=0.0010, ngram_range=[1,3]).fit_transform(data["text"]).todense()

In [17]:
tfVect=None

In [20]:
tfVect.shape

(96269, 7578)

In [257]:
from scipy import spatial
import operator


def sim_rank(tx_id,tf_corpus, top=3):
    all_score={}
    
    for i in range(len(tf_corpus)):
        if i==tx_id:
            #print(data.iloc[i]["text"])
            continue
        score= result = 1 - spatial.distance.cosine(tf_corpus[tx_id], tf_corpus[i])
        all_score[i] = score
    sorted_scores = sorted(all_score.items(), key=operator.itemgetter(1))
    sorted_scores.reverse()
    
#     print(data["text"][tx_id])
#     print(data["text"][sorted_scores[0][0]])
#     print(data["text"][sorted_scores[1][0]])
#     print(data["text"][sorted_scores[2][0]])
    return sorted_scores[0:top]
    
        
    
    
    

In [191]:
all_ranks=sim_rank(353,tfVect,top=50)

fallen angel redeem bibl devil fallen angel creat good chose join lucif rebel consequ lost place heaven angel save angel remain choos chang side thing salvat angel


In [233]:
l1=data["text"][353].split(" ")

print(l1, len(l1))

['fallen', 'angel', 'redeem', 'bibl', 'devil', 'fallen', 'angel', 'creat', 'good', 'chose', 'join', 'lucif', 'rebel', 'consequ', 'lost', 'place', 'heaven', 'angel', 'save', 'angel', 'remain', 'choos', 'chang', 'side', 'thing', 'salvat', 'angel'] 27


In [234]:
l2=data["text"][79261].split(" ")
print(l2, len(l2))

['bibl', 'devil', 'fallen', 'angel', 'creat', 'good', 'chose', 'join', 'lucif', 'rebel', 'consequ', 'lost', 'place', 'heaven', 'angel', 'save', 'angel', 'remain', 'choos', 'chang', 'side', 'thing', 'salvat', 'angel'] 24


In [235]:
print(list(set(l1).intersection(l2)))
print(len(list(set(l1).intersection(l2))))

['heaven', 'save', 'lost', 'angel', 'rebel', 'good', 'bibl', 'consequ', 'thing', 'place', 'side', 'choos', 'chang', 'fallen', 'chose', 'salvat', 'join', 'devil', 'creat', 'lucif', 'remain']
21


In [114]:
data["text"][15969]

'differ type cathol christian call roman cathol actual latin rite cathol eastern orthodox made rest call latin rite eastern rite older'

In [305]:
data.iloc[1]["label"]

0

In [196]:
all_ranked_labels=[]
print(all_ranks)
for k in range(len(all_ranks)):
    
    all_ranked_labels.append(data.iloc[all_ranks[k][0]]["label"])
    

[(74840, 1.0), (72516, 1.0), (79261, 0.9676458882544372), (1989, 0.7354877758670498), (88353, 0.6922723154639895), (61574, 0.677995199001079), (64653, 0.6571531322849749), (3728, 0.6318239394250972), (19678, 0.6189904721698241), (19680, 0.6169877132252042), (35426, 0.6136494253746404), (92248, 0.6017633128070966), (92247, 0.5874486592304566), (16299, 0.5697745311797408), (42641, 0.5563792643758267), (56131, 0.5522454016664318), (57539, 0.5420826163803141), (44082, 0.5381335830135789), (290, 0.5303476883100257), (70384, 0.5261336147738995), (9090, 0.5227128550531639), (20737, 0.5144285573837044), (85958, 0.5141667439807481), (77223, 0.5141667439807481), (47296, 0.5131008298213231), (59058, 0.5128983371786783), (14256, 0.5114020692793712), (37093, 0.510058656133014), (30383, 0.5066956266309778), (2246, 0.5012466768121568), (55643, 0.4998145458516212), (75423, 0.49784045179539804), (4681, 0.49784045179539804), (62286, 0.4965798189883486), (19682, 0.49427915176566184), (19530, 0.4929372321

In [194]:
all_ranked_labels

[2,
 2,
 2,
 3,
 8,
 0,
 3,
 2,
 0,
 0,
 3,
 0,
 0,
 8,
 2,
 2,
 2,
 3,
 3,
 2,
 0,
 0,
 3,
 3,
 2,
 3,
 0,
 2,
 3,
 2,
 2,
 3,
 3,
 8,
 0,
 0,
 3,
 3,
 0,
 0,
 2,
 3,
 3,
 3,
 2,
 2,
 3,
 2,
 2,
 0]

In [195]:
from collections import Counter
print(Counter(all_ranked_labels).keys())
print(Counter(all_ranked_labels).values())

dict_keys([8, 0, 2, 3])
dict_values([3, 13, 17, 17])


In [138]:
y

<1x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 49 stored elements in Compressed Sparse Row format>

In [139]:
classifier.predict(x_Test[0])

array([1], dtype=int64)

In [258]:

def sim_score(tx_id,tf_corpus, top=3):
    #print("true label - {0}".format(data.iloc[tx_id]["label"]))
    trueL= data.iloc[tx_id]["label"]
    all_ranks=sim_rank(tx_id,tfVect,top=top)
     
    all_ranked_labels=[]
    cumu_scores={}
    for k in range(len(all_ranks)):
        if data.iloc[all_ranks[k][0]]["label"] not in cumu_scores:
            cumu_scores[data.iloc[all_ranks[k][0]]["label"]] = all_ranks[k][1]
        else:
            cumu_scores[data.iloc[all_ranks[k][0]]["label"]] = cumu_scores[data.iloc[all_ranks[k][0]]["label"]] + all_ranks[k][1]
           
        all_ranked_labels.append(data.iloc[all_ranks[k][0]]["label"])
        keys= Counter(all_ranked_labels).keys()
        vals= Counter(all_ranked_labels).values()
   
#     print(all_ranks)
    finalscore= sorted(cumu_scores.items(), key=operator.itemgetter(1))
    finalscore.reverse()
    #print("Predicted label- {0}".format(finalscore[0]))
    #print(cumu_scores, keys, vals) 
    return trueL, finalscore[0][0]
    
    

In [253]:
tl, pl=sim_score(400,tfVect,top=50)

bibl talk wine alcohol wine discuss past accept christian drink alcohol heard dri believ wine bibl wine common today actual alcohol grape juic person believ wine jesus day wine biblic histor evid contrari wine past ferment juic
Predicted label- (9, 24.484298493779182)
{0: 6.761262313498027, 9: 24.484298493779182} dict_keys([0, 9]) dict_values([12, 38])


In [255]:
print(tl,pl[0])

9 9


In [264]:
import datetime
labelt=[]
labelp=[]
for i in range(50):
    print("iteration -{0} at {1}".format(i, datetime.datetime.now()))
    tl, pl=sim_score(i,tfVect,top=50)
    labelt.append(tl)
    labelp.append(pl)
    
    

iteration -0 at 2018-11-19 22:17:13.192874
iteration -1 at 2018-11-19 22:17:42.741864
iteration -2 at 2018-11-19 22:18:13.886592
iteration -3 at 2018-11-19 22:18:44.334181
iteration -4 at 2018-11-19 22:19:13.104273
iteration -5 at 2018-11-19 22:19:43.894945
iteration -6 at 2018-11-19 22:20:17.272706
iteration -7 at 2018-11-19 22:20:48.227935
iteration -8 at 2018-11-19 22:21:18.111038
iteration -9 at 2018-11-19 22:21:49.368459
iteration -10 at 2018-11-19 22:22:19.256544
iteration -11 at 2018-11-19 22:22:49.265348
iteration -12 at 2018-11-19 22:23:19.452602
iteration -13 at 2018-11-19 22:23:47.544580
iteration -14 at 2018-11-19 22:24:15.917714
iteration -15 at 2018-11-19 22:24:45.804802
iteration -16 at 2018-11-19 22:25:14.693597
iteration -17 at 2018-11-19 22:25:44.036145
iteration -18 at 2018-11-19 22:26:13.645933
iteration -19 at 2018-11-19 22:26:43.251816
iteration -20 at 2018-11-19 22:27:05.307833
iteration -21 at 2018-11-19 22:27:35.509120
iteration -22 at 2018-11-19 22:28:01.02485

In [325]:
#Take random ranges of documents to predict.
for i in range(35000, 35050):
    print("iteration -{0} at {1}".format(i, datetime.datetime.now()))
    tl, pl=sim_score(i,tfVect,top=50)
    labelt.append(tl)
    labelp.append(pl)

iteration -35000 at 2018-11-20 19:20:04.791589
iteration -35001 at 2018-11-20 19:20:40.101350
iteration -35002 at 2018-11-20 19:21:14.359753
iteration -35003 at 2018-11-20 19:21:46.422023
iteration -35004 at 2018-11-20 19:22:18.570065
iteration -35005 at 2018-11-20 19:22:54.659570
iteration -35006 at 2018-11-20 19:23:33.908625
iteration -35007 at 2018-11-20 19:24:12.574242
iteration -35008 at 2018-11-20 19:24:49.194327
iteration -35009 at 2018-11-20 19:25:28.561069
iteration -35010 at 2018-11-20 19:26:14.541128
iteration -35011 at 2018-11-20 19:27:03.439385
iteration -35012 at 2018-11-20 19:27:46.325727
iteration -35013 at 2018-11-20 19:28:35.695756
iteration -35014 at 2018-11-20 19:29:07.234426
iteration -35015 at 2018-11-20 19:29:38.245509
iteration -35016 at 2018-11-20 19:30:08.728021
iteration -35017 at 2018-11-20 19:30:40.217808
iteration -35018 at 2018-11-20 19:31:10.616734
iteration -35019 at 2018-11-20 19:31:40.743129
iteration -35020 at 2018-11-20 19:32:10.530484
iteration -35

In [284]:
len(labelt)

271

In [327]:
from sklearn.metrics import precision_score, f1_score,recall_score, accuracy_score
accuracy_score(labelt, labelp)

0.5856697819314641